In [ ]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
import time

glueContext = GlueContext(SparkContext.getOrCreate())

In [ ]:
fooddf = glueContext.create_dynamic_frame.from_catalog(
    database="dojodb",
    table_name="postgres_public_fooddemand",
)

In [ ]:
fooddf1 = glueContext.create_dynamic_frame.from_catalog(
    database="dojodb",
    table_name="postgres_public_fooddemand",
    additional_options = {
            'hashfield': 'week',
            'hashpartitions': '5'
    }
)

In [ ]:
fooddf.count()

In [ ]:
fooddf1.count()

In [ ]:
import boto3
import json

secret_name = "<SCRETS_ARN>"
region_name = "<REGION>"
session = boto3.session.Session()
client = session.client(service_name='secretsmanager',region_name=region_name)

response = client.get_secret_value(SecretId=secret_name)

secrets = json.loads(response["SecretString"])

uname = secrets["username"]
pwd = secrets["password"]
host = secrets["host"]

print(host)

In [ ]:
glueContext.write_dynamic_frame.from_options(
    frame=fooddf,
    connection_type="redshift",
    connection_options = {
        "user" : uname,
        "password" : pwd,
        "url" : "jdbc:redshift://" + host + ":5439/dev",
        "dbtable" : "public.fooddemand",
        "redshiftTmpDir": "s3://dojo-dataset/temp/",
        "bulkSize" : "2"} 
)